In [1]:
from surprise import Dataset, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV

In [2]:
data = Dataset.load_builtin('ml-100k')

In [3]:
param_grid_svd = {
    'n_epochs': [5, 10, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6] 
}

In [4]:
param_grid_nmf = {
    'n_factors': [15, 20],
    'n_epochs': [50, 70],
    'reg_pu': [0.06, 0.08],
    'reg_qi': [0.06, 0.08]
}

In [5]:
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse'], cv=3, n_jobs=-1)
gs_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse'], cv=3,  n_jobs=-1)

In [6]:
%%time
gs_svd.fit(data)

CPU times: total: 4.92 s
Wall time: 9.22 s


In [7]:
%%time
gs_nmf.fit(data)

CPU times: total: 6.56 s
Wall time: 19.2 s


In [8]:
best_params_svd = gs_svd.best_params['rmse']
best_params_nmf = gs_nmf.best_params['rmse']
print(f"Best parameters SVD: {best_params_svd}")
print(f"Best parameters NMF: {best_params_nmf}")

Best parameters SVD: {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4}
Best parameters NMF: {'n_factors': 20, 'n_epochs': 70, 'reg_pu': 0.08, 'reg_qi': 0.08}


In [9]:
model_svd = SVD(**best_params_svd)
model_svdpp = SVDpp(**best_params_svd)
model_nmf = NMF(**best_params_nmf)

In [10]:
%%time
cross_validate(model_svd, data, measures=['RMSE'], cv=5,n_jobs=-1,  verbose=True);

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9564  0.9581  0.9627  0.9521  0.9566  0.9572  0.0034  
Fit time          0.99    1.04    1.06    1.03    1.02    1.03    0.02    
Test time         0.28    0.30    0.29    0.29    0.27    0.29    0.01    
CPU times: total: 1.3 s
Wall time: 2.63 s


In [11]:
%%time
cross_validate(model_svdpp, data, measures=['RMSE'], cv=5, n_jobs=-1, verbose=True);

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9516  0.9512  0.9585  0.9634  0.9573  0.9564  0.0045  
Fit time          41.99   42.39   42.04   42.14   42.20   42.15   0.14    
Test time         5.66    5.54    5.58    5.63    5.47    5.58    0.07    
CPU times: total: 1.3 s
Wall time: 49 s


In [12]:
%%time
cross_validate(model_nmf, data, measures=['RMSE'], cv=5, n_jobs=-1, verbose=True);

Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9424  0.9528  0.9451  0.9490  0.9536  0.9486  0.0043  
Fit time          2.14    2.22    2.13    2.14    2.10    2.15    0.04    
Test time         0.24    0.22    0.23    0.22    0.22    0.22    0.01    
CPU times: total: 1.34 s
Wall time: 3.61 s


# Висновок
Всі три алгоритми показують низьке стандартне відхилення результатів RMSE (приблизно від 0,0057 до 0,0062), що свідчить про стабільну продуктивність на різних ділянках тесту. NMF та SVD мають відносно схожий і короткий час тестування, близько 0,21 і 0,30 секунди відповідно. SVDpp займає більше часу, середній час тесту становить 6,00 секунд. Таким чином, якщо пріоритетом є точність прогнозування, NMF може бути кращим вибором, за ним слідує SVD. Однак, якщо часова ефективність є критичним фактором, SVD виділяється як більш ефективний вибір, пропонуючи хороший баланс між точністю і швидкістю. SVDpp, хоча і є конкурентоспроможним за точністю, не може бути ідеальним для сценаріїв, де ефективність використання часу має вирішальне значення через довший час тренування і тестування.